Zalecamy nie czytać notatników na githubie, ze względu na źle wyświetlające się wizualizacje i brak możliwości uruchamiania kodu. Polecamy otworzyć notatnik w google colab.

# **Olimpiada AI - kurs wprowadzający 2025 - Wykład 05A**

"Wstęp do Regresji Liniowej"


## Regresja Liniowa - Intuicja

Cała istota Uczenia Maszynowego polega na szukaniu "ukrytych" funkcji, które przekształcają nam jedną zmienną (wejściową) w drugą (wyjściową). Wyobraźcie sobie, że chcemy przewidzieć wynik ze sprawdzianu (zmienna wyjściowa) z matematyki na podstawie czasu poświęconego na naukę (zmienna wejściowa).

W liceum uczyliście się na pewno wzoru na funkcję liniową:
$$y = ax + b$$

Teraz wyobraźmy sobie, że to jes prawdziwa funkcją opisująca ile czasu trzeba poświęcić na naukę, gdzie:

  * $y$ to wynik ze sprawdzianu
  * $x$ to nasza dana wejściowa - godziny nauki
  * $a$ i $b$ to parametry (współczynniki), których szukamy

Stwórzmy przykładowe dane:

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Ustawiamy ziarno losowości, żeby każdy miał ten sam wynik
torch.manual_seed(2025)

def generate_student_data(n=50):
    # Generujemy losowe godziny nauki (od 0 do 100 godzin)
    hours_studied = torch.rand(n) * 100

    # Prawdziwa zależność (ukryta przed modelem):
    # Każda godzina nauki daje średnio 0.8 punktu, startujemy z poziomu 10pkt
    scores = 0.8 * hours_studied + 10

    # Dodajemy "szum": stres, szczęście, dyspozycja dnia (+/- 15pkt)
    # noise = torch.randn(n) * 15
    # scores += noise

    return hours_studied, scores

X_hours, y_scores = generate_student_data()

plt.figure(figsize=(8, 6))
plt.scatter(X_hours, y_scores, color='blue', label='Wyniki różnych uczniów')
plt.title("Zależność: Czas nauki a wynik ze sprawdzianu z matematyki")
plt.xlabel("Godziny nauki (x)")
plt.ylabel("Wynik % (y)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

Spróbujmy teraz wykonać bardzo prostą esymację - po prostu przypisać każdemu uczniowi średnią wartość z testu

In [ ]:
def mean_estimate(a_guess, b_guess, X, y, plot_error=False):
    # Przewidywania naszej "zgadywanej" linii
    y_pred = a_guess * X + b_guess

    plt.figure(figsize=(10, 6))
    plt.scatter(X, y, color='blue', label='Dane prawdziwe')
    plt.plot(X, y_pred, color='red', linewidth=2, label=f'Nasza linia: y = {a_guess}x + {b_guess}')

    # Rysowanie błędów (Reszt)


    total_error = 0
    for i in range(len(X)):
        # Rysujemy pionową kreskę od punktu do linii
        if plot_error:
            plt.plot([X[i], X[i]], [y[i], y_pred[i]], 'g--', alpha=0.5)
        # Sumujemy błędy (kwadrat różnicy, żeby unikać liczb ujemnych)
        total_error += abs(y[i]-y_pred[i])

    plt.title(f"Wizualizacja błędu (Loss). Suma błędów: {total_error:.0f}")
    plt.xlabel("Godziny nauki")
    plt.ylabel("Wynik (w punktach)")
    plt.legend()
    plt.show()

# Spróbujmy zgadnąć: a=0 (brak wpływu nauki), b=50 (każdy dostaje 50pkt)
mean_estimate(a_guess=0.0, b_guess=50, X=X_hours, y=y_scores, plot_error=False)

## Jak dobra jest nasza estymacja?

My widzimy na oko, czy linia jest dobra czy nie i gdzie mniej-więcej powinna przebiegać idealna linia. Komputer tego nie widzi. Komputer potrafi tylko liczyć wartości i na ich podstawie podejmować "decyzje".

W Machine Learningu ocenę, jak dobra jest nasza estymacja nazywamy "Funkcją Kosztu" (Loss Function). W tym przypadku to po prostu suma błędów - odległości od poszczególnych pomiarów - jakie popełnia nasza estymacja (linia).

**Ważne** - zauważ, że tutaj odległość od punktów liczymy tylko względem osi Y, to nie jest odległość liczona pod kątem prostym do naszej estymacji! 

Sprawdźmy jak to wygląda na przykładzie bardzo prostej (i niezbyt przydatnej estymacji).

In [ ]:
mean_estimate(a_guess=0.0, b_guess=50, X=X_hours, y=y_scores, plot_error=True)

Zielone kreski to błędy. Celem algorytmów uczących jest tak manipulować `a` i `b`, żeby suma długości zielonych kresek była jak najmniejsza.

## Machine Learning w akcji (PyTorch)

Teraz użyjemy biblioteki *PyTorch*, aby komputer sam znalazł idealną linię. Użyjemy metody zwanej Gradient Descent (spadek wzdłuż gradientu) o której powiemy sobie więcej za chwilę.

In [ ]:
import torch.nn as nn

# 1. Przygotowanie danych (format wymagany przez PyTorch)
X_tensor = X_hours.view(-1, 1) # Zmieniamy kształt na kolumnę
y_tensor = y_scores.view(-1, 1)

# 2. Definiujemy parametry a i b (na początku losowe!)
# requires_grad=True oznacza: "PyTorch, śledź te zmienne i mów mi, jak je zmieniać"
a = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

# 3. Ustawienia uczenia
learning_rate = 0.0001 # Jak duże kroki robimy
optimizer = torch.optim.SGD([a, b], lr=learning_rate)
loss_fn = nn.MSELoss()

print("Startujemy naukę...")
print(f"Początkowe losowe parametry: a = {a.item():.2f}, b = {b.item():.2f}")

# 4. Pętla uczenia (Trening)
errors_history = []

for epoch in range(1000):
    # Krok 1: Model przewiduje wynik
    y_pred = a * X_tensor + b

    # Krok 2: Liczymy błąd (Loss)
    loss = loss_fn(y_pred, y_tensor)
    errors_history.append(loss.item())

    # Krok 3: Gradient Backward (na razie magia, opowiemy sobie o tym za chwilę)
    # PyTorch liczy pochodne - sprawdza w którą stronę zmienić a i b, żeby błąd zmalał
    optimizer.zero_grad()
    loss.backward()

    # Krok 4: Aktualizacja parametrów
    optimizer.step()

    if epoch % 200 == 0:
        print(f"Epoka {epoch}: Błąd średni = {loss.item():.2f}")

print("-" * 30)
print(f"NAUCZONE parametry: a = {a.item():.2f}, b = {b.item():.2f}")
print("(Pamiętacie? Prawdziwe ukryte 'a' wynosiło 0.8, a 'b' 0.1)")

### Sprawdźmy wynik graficznie

In [ ]:
# Wizualizacja po treningu
mean_estimate(a_guess=a.item(), b_guess=b.item(), X=X_hours, y=y_scores, plot_error=True)

## Wielomiany

Regresja liniowa zakłada, że funkcja, którą modelujemy to prosta. Ale wyobraźcie sobie na przykład drogę hamowania samochodu albo tor lotu piłki - to są przykłady krzywych.

Jeśli użyjemy zwykłej linii do opisania zakrętu, nasz błąd będzie bardzo duży, dlatego musimy sięgnąć po trochę potężniejsze narzędzie. Żeby przybliżyć krzywą za pomocą regresji liniowej musimy użyć pewnego tricku - do naszej estymacji dodajemy potęgi $x^2, x^3...$. Algorytm jest ten sam, to wciąż regresja, ale tworzy nam Wielomian.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Dane: Czas (t) vs Pozycja samochodu
t = np.linspace(0, 1, 100).reshape(-1, 1)

# Prawdziwy ruch jest skomplikowany (wielomian)
# Wyobraźcie sobie, że samochód przyspiesza i skręca
y_position = t**3 - 0.5 * t**2 + 0.2 * t
y_position += np.random.normal(0, 0.02, size=y_position.shape) # Szum

# 1. Próba dopasowania PROSTEJ LINII
model_linear = LinearRegression()
model_linear.fit(t, y_position)
y_pred_line = model_linear.predict(t)

# 2. Próba dopasowania WIELOMIANU
# Zamieniamy zwykłe [t] na [t, t^2, t^3]
poly = PolynomialFeatures(degree=3)
t_poly = poly.fit_transform(t)

model_poly = LinearRegression()
model_poly.fit(t_poly, y_position)
y_pred_poly = model_poly.predict(t_poly)

# Wykres
plt.figure(figsize=(10, 6))
plt.scatter(t, y_position, color='gray', s=10, label='Zaszumione odczyty pozycji z drogi hamowania')
plt.plot(t, y_pred_line, color='red', linestyle='--', label='Model Liniowy')
plt.plot(t, y_pred_poly, color='green', linewidth=3, label='Model Wielomianowy')

plt.title("Dlaczego potrzebujemy bardziej ekspresywnych modeli?")
plt.xlabel("Czas")
plt.ylabel("Pozycja")
plt.legend()
plt.show()

## Dlaczego potrzebujemy bardziej ekspresyjnych modeli?

W praktyce, takie funkcje są dużo bardziej skomplikowane niż proste lub nawet wielomiany. Każdy problem w uczeniu maszynowym próbujemy rozwiązać funkcją! Nawet klasyfikcaję obrazów czy generowanie tekstów za pomocą ChatuGPT. Dlatego gdy trenujemy ogromne sieci neuronowe (jak ChatGPT czy systemy rozpoznawania twarzy), one są trenowane tak samo jak widzieliście tutaj. Próbują estymować funkcję, która za pomocą danych wejściowych chce przewidzieć dane wyjściowe. To znaczy, że funkcje *przekształcają przestrzeń* wejściową, tak żeby uzyskać wyjściową.